# Business Problem

The Tanzania Development Trust is a UK charitable organization operating within the country of Tanzania since 1975.

They focus on development in rural Tanzania, aiming to support small projects in the poorest parts of the country where one of their priority areas of funding is clean water. Their stated water project involves boreholing and rope pump installation in areas with limited access to clean water, currently located in the regions of Kagera and Kigoma in the northwest of the country.

A new benefactor wants to expand the project not only geographically to more of the country, but in the scope of repairing existing pumps before they fail. I have been tasked with developing a model to predict the operating condition of a current waterpoint: functional, needs repair, or non-functional.

The main objective is to identify waterpoints that are in need of repair. [Research shows](https://sswm.info/entrepreneurship-resource/developing-impactful-businesses/maintenance-services-for-rural-water-pumps) that it is much less expensive to repair and rehabilitate a waterpoint, as well as being more protective of the water resources in the country. 
The secondary objective is to identify concentrations of non-functioning water points that may be an eligible location for a new installation.
______________________
The data provided for modeling was collected between March 2011 and March 2013, and contains the information for 59,400 water points 

# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import training data and target
raw_data = pd.read_csv('data/training_data.csv')
raw_target = pd.read_csv('data/training_target.csv')

display(raw_target.info())
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Create a raw dataframe with merged data and target. We will use this df during initial EDA so we can compare feature relationships with target, and so we can understand and deal with null values

In [3]:
raw_df = pd.merge(raw_data, raw_target, on='id')

The dataset for training includes 59,400 entries with 39 total features, a unique identifier, and a target label.

In [4]:
status_values = pd.DataFrame(raw_df.status_group.value_counts())
status_values['percent'] = round(raw_df.status_group.value_counts(normalize=True) * 100, 1)
status_values

,status_group,percent
functional,32259,54.3
non functional,22824,38.4
functional needs repair,4317,7.3


This is a ternary classification problem. The three possible values are:
- functional (F)
- non functional (NF)
- functional needs repair (FR)

Value counts show that our dataset is not balanced with respect to the label values. Only 7.3% of pumps are classified as functional needs repair, while 54.3% are functional and 38.4% are non functional. We will need to keep this imbalance in mind when modeling.
_________
Before any modeling can occur we must check and deal with null values


# Null Checks

In [5]:
null_checks = pd.DataFrame(data=raw_df.isna().sum(),
                          columns=['null_count'])
null_checks['percent_of_data'] = round((null_checks.null_count / len(raw_data)) * 100, 1)
null_checks = null_checks[null_checks.percent_of_data > 0.1]
null_checks.sort_values('percent_of_data', ascending=False, inplace=True)
null_checks

,null_count,percent_of_data
scheme_name,28166,47.4
scheme_management,3877,6.5
installer,3655,6.2
funder,3635,6.1
public_meeting,3334,5.6
permit,3056,5.1
subvillage,371,0.6


There are 7 features with null values in our dataset, and we can what that number of nulls is by percent of total available data. 
______
All of the features that contain null values are object types and will need to be converted. Before conversion we will need to address the null values.

## subvillage

In [6]:
subvillage_nans = raw_df[raw_df.subvillage.isnull()]
round(subvillage_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 55.26
non functional             44.47
functional needs repair     0.27
Name: status_group, dtype: float64

The null values in subvillage represent 0.6% of our total data. The distribution of the target label is close to the whole dataset

In [7]:
subvillage_nans.region.value_counts()

Dodoma    361
Mwanza     10
Name: region, dtype: int64

All but 10 of our subvillage nan's come from the region of Dodoma, the rest from Mwanza. Lets look at the subvillage distribution of those regions from the whole dataset

In [8]:
raw_df[raw_df['region'] == 'Dodoma'].subvillage.value_counts()

Kawawa         54
Shuleni        43
Nyerere        35
Azimio         34
Majengo        32
               ..
Foye            1
Mtatangwe       1
Makao Mapya     1
Soya Mjini      1
Mgomwa          1
Name: subvillage, Length: 705, dtype: int64

In [9]:
raw_df[raw_df['region'] == 'Mwanza'].subvillage.value_counts()

1                     132
Madukani               52
Bujingwa               25
Shuleni                19
Matale                 18
                     ... 
Bukalo                  1
Nyambona                1
Kabaganda B             1
Bulyahilu Center B      1
Mwambogwa               1
Name: subvillage, Length: 1507, dtype: int64

There are no average or overwhelmingly dominant subvillages that we could assign the null values to. It's not clear if we will use subvillage in modeling, so we will change null values to 'Other'

In [10]:
raw_df['subvillage'].fillna(value='Other', inplace=True)

## permit

In [11]:
permit_nans = raw_df[raw_df.permit.isnull()]
permit_nans.reset_index(drop=True, inplace=True)
round(permit_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 54.74
non functional             35.44
functional needs repair     9.82
Name: status_group, dtype: float64

5% of our dataset have no value for permit. Distribution of the target label is approximately the same as the whole dataset.

In [12]:
permit_distribution = raw_df.permit.value_counts(normalize=True)
permit_distribution

True     0.68955
False    0.31045
Name: permit, dtype: float64

Per the data documentation, the permit feature is if the water point is permitted or not. Data we do have for this feature show it's about 70/30 in favor of permitted.

We will randomly fill these 3056 missing datapoints with true/false in the same ratio we found in our entire dataset.

In [13]:
raw_df['permit'] = raw_df['permit'].fillna(pd.Series(np.random.choice([True, False],
                                                       p=list(permit_distribution),
                                                       size=len(raw_df))))

## public_meeting

In [14]:
public_meeting_nans = raw_df[raw_df.public_meeting.isnull()]
public_meeting_nans.reset_index(drop=True, inplace=True)
round(public_meeting_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 50.33
non functional             44.99
functional needs repair     4.68
Name: status_group, dtype: float64

5.6% of our dataset has no value for public_meeting. Distribution of the target label is approximately the same as the whole dataset.

In [15]:
meeting_distribution = raw_df.public_meeting.value_counts(normalize=True)
meeting_distribution

True     0.909838
False    0.090162
Name: public_meeting, dtype: float64

The public meeting feature is a boolean that is over 90% true for data we do have. We will fill null values in the same percentages.

In [16]:
raw_df['public_meeting'] = raw_df['public_meeting'].fillna(
    pd.Series(np.random.choice([True, False],
                               p=list(meeting_distribution),
                               size=len(raw_df))))

## funder & installer

The features 'funder' and 'installer' have almost the same number of null values; I am curious about the overlap of nulls.

In [17]:
# dividing the number of entries with null for both features by the smaller count
len(raw_df[raw_df.funder.isnull() & raw_df.installer.isnull()]) / null_checks.null_count['funder']

0.9854195323246218

Over 98% of the null values for funder also contain null values for installer.
________

In [18]:
funder_df = pd.DataFrame(round(raw_df.funder.value_counts(normalize=True, dropna=False) * 100, 2))
funder_df

,funder
Government Of Tanzania,15.29
NaN,6.12
Danida,5.24
Hesawa,3.71
Rwssp,2.31
...,...
Rarymond Ekura,0.00
Justine Marwa,0.00
Municipal Council,0.00
Afdp,0.00


In [19]:
funder_df[funder_df.funder > 1.0].sum()


funder    52.69
dtype: float64

Including null values, there were 1,898 distinct values for funder. Of that, 18 values (including null) have representative counts more than 1% of total data.

Those 18 distinct values represent almost 53% of our total data. We will convert null values to 'Other'. An additional consideration is binning everything less than 1% total share of funder as 'Other' to reduce the unique value count.

In [20]:
installer_df = pd.DataFrame(round(raw_df.installer.value_counts(normalize=True, dropna=False) * 100, 2))
installer_df

,installer
DWE,29.30
NaN,6.15
Government,3.07
RWE,2.03
Commu,1.78
...,...
Wizara ya maji,0.00
TWESS,0.00
Nasan workers,0.00
R,0.00


In [21]:
installer_df[installer_df.installer > 1.0].sum()

installer    51.6
dtype: float64

Similarly to funder, the installer feature is dominated by small share installers. Of the 2,146 distinct values for installer, 12 values (including null) have representative counts more than 1% of total data.

Those 12 distinct values represent almost 52% of our total data. This is similar to the funder feature. We will also convert null values to 'Other', and will consider binning all installers with less than 1% total share of installer as 'Other' to reduce the unique value count. 

In [22]:
raw_df['funder'].fillna(value='Other', inplace=True)
raw_df['installer'].fillna(value='Other', inplace=True)

## scheme_management

In [23]:
scheme_management_nans = raw_df[raw_df.scheme_management.isnull()]
scheme_management_nans.reset_index(drop=True, inplace=True)
round(scheme_management_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 48.31
non functional             45.94
functional needs repair     5.75
Name: status_group, dtype: float64

6.5% of our data has no value for scheme_management. Distribution of the target data is approximately the same as the whole dataset.

We will fill null values for scheme_management with other values from the feature in the same percentage. 

There was only one entry with the value of 'None', we will change that to 'Other'

In [29]:
raw_df.at[23603, 'scheme_management'] = 'Other'

In [37]:
scheme_management_list = pd.DataFrame(raw_df.scheme_management.value_counts(normalize=True))
scheme_management_list
# scheme_management_list['probabilities'] = scheme_management_list.scheme_management.astype('float64')

,scheme_management
VWC,0.662662
WUG,0.093763
Water authority,0.056787
WUA,0.051924
Water Board,0.049493
Parastatal,0.030258
Private operator,0.019145
Company,0.019109
Other,0.013814
SWC,0.001747


In [40]:
raw_df['scheme_management'] = raw_df['scheme_management'].fillna(
    pd.Series(np.random.choice(list(scheme_management_list.index),
                               p=list(scheme_management_list.scheme_management),
                               size=len(raw_df))))

In [58]:
raw_df.scheme_management.value_counts(normalize=True)

VWC                 0.661835
WUG                 0.093889
Water authority     0.056751
WUA                 0.052391
Water Board         0.049529
Parastatal          0.030202
Company             0.019125
Private operator    0.019040
Other               0.014091
SWC                 0.001852
Trust               0.001296
Name: scheme_management, dtype: float64

## scheme_name

In [57]:
raw_df.scheme_name.value_counts(normalize=True, dropna=False)

NaN                        0.474175
K                          0.011481
None                       0.010842
Borehole                   0.009192
Chalinze wate              0.006818
                             ...   
Visiga water supplly       0.000017
Emanyata pipelines         0.000017
Magundi water supply       0.000017
Imalampaka water supply    0.000017
Mtawanya                   0.000017
Name: scheme_name, Length: 2697, dtype: float64

Almost half (47%) of the scheme_name feature contains no data, and the remaining data contains 2,697 distinct other features, none of which exceed 1% of the dataset. Lets look a little closer to see what else we can figure out.

In [56]:
scheme_name = pd.DataFrame(raw_df.scheme_name.value_counts(dropna=False))
scheme_name[scheme_name.scheme_name > 75]

,scheme_name
NaN,28166
K,682
None,644
Borehole,546
Chalinze wate,405
M,400
DANIDA,379
Government,320
Ngana water supplied scheme,270
wanging'ombe water supply s,261


The scheme name, per the documentation, is the individual or group that actually operates the waterpoint. This is compared to the scheme management company, which oversees operation. When the data was collected, it looks like there was little organization with respect to this particular datapoint. Considering the there are no other patterns in the available names, and that we have the management data, we will drop this feature from our dataset.

In [59]:
raw_df.drop('scheme_name', axis=1, inplace=True)

# Exploring numerical data

# Exploring extraction types

In [ ]:
extraction_types = pd.DataFrame(raw_df.extraction_type.value_counts())
extraction_types.columns = ['total_count']
extraction_types['percent_of_total'] = round((extraction_types.total_count / len(raw_df)) * 100, 2)
extraction_types['non-functional'] = raw_df[raw_df.status_group == 'non functional'].extraction_type.value_counts()
extraction_types['non-functional_percent'] = round((extraction_types['non-functional'] / extraction_types.total_count) * 100, 2)
extraction_types['needs_repair'] = raw_df[raw_df.status_group == 'functional needs repair'].extraction_type.value_counts()
extraction_types.fillna(0.0, inplace=True)
extraction_types['needs_repair'] = extraction_types['needs_repair'].astype('int')
extraction_types['needs_repair_percent'] = round((extraction_types['needs_repair'] / extraction_types.total_count) * 100, 2)
extraction_types